# Jupyter notebook on remote server

Jupyter notebook works very well with remote servers. That's important for earth science modeling people because we seldom run models and store data on our personal computers!

Traditionally, we display figures on a remote server using [x11 forwarding](https://en.wikipedia.org/wiki/X_Window_System), but that's extremely slow and often annoying. On the contrary, Jupyter simply uses HTTP protocal and runs in a web browser. You user experience will not change at all when using Jupyter on a remote server - still a web browser, still very fast.

## Private server

On your own server or cloud computing platforms, setting up Jupyter is almost trivial. See the following link or other similar tutorials.

- https://techtalktone.wordpress.com/2017/03/28/running-jupyter-notebooks-on-a-remote-server-via-ssh/

## Shared HPC cluster

### General reference

The way to set up Jupyter on a shared HPC cluster could be system-dependent. You may ask system architects or refer to:

- https://medium.com/@rabernat/custom-conda-environments-for-data-science-on-hpc-clusters-32d58c63aa95
- http://ipyrad.readthedocs.io/HPC_Tunnel.html

### Harvard Odyssey

The following script works on Harvard Odyssey, which uses the slurm system.

    #!/bin/bash
    #SBATCH -p general
    #SBATCH -N 1
    #SBATCH -c 1
    #SBATCH -t 2:00:00
    #SBATCH --mem-per-cpu 8000
    #SBATCH --job-name notebook
    #SBATCH --output jupyter-address-%J.log

    ## get tunneling info
    XDG_RUNTIME_DIR=""
    ipnport=$(shuf -i8000-9999 -n1)
    ipnip=$(hostname -i)

    ## print tunneling instructions to the output file
    echo -e "
        Copy/Paste this in your local terminal to ssh tunnel with remote
        -----------------------------------------------------------------
        ssh -N -L $ipnport:$ipnip:$ipnport user@host
        -----------------------------------------------------------------

        Then open a browser on your local machine to the following address
        ------------------------------------------------------------------
        localhost:$ipnport  (prefix w/ https:// if using password)
        ------------------------------------------------------------------
        "

    ## start an ipcluster instance and launch jupyter server
    source activate GCPy # change it to your environment name
    jupyter-notebook --NotebookApp.token='' --no-browser --port=$ipnport --ip=$ipnip